# Run application

In this page I will introduce how to run `fastapi` applications in general and in particular how to run examples related to `fastapi` in the site.

<a href="https://fastapi.tiangolo.com/deployment/docker/">Here</a> you can find out more details about using `fastapi` in the docker.

## dockerfile{#sec-dockerfile}

In the next cell is the docker file I am using for this example.

In [9]:
%%writefile run_application_files/dockerfile
FROM python:3.11
COPY requrements.txt requrements.txt
RUN pip3 install -r requrements.txt
EXPOSE 8000

## requrements.txt

Python libraries you only needed to run the `fastapi` server. It is supposed to be used in the [dockerfile](#sec-dockerfile) described above.

In [6]:
%%writefile run_application_files/requrements.txt
fastapi==0.103.1
uvicorn==0.23.2

## Programme{#sec-prog}

You need to declare an object of class `fastapi.fastAPI`. Then use its decorators to add to your functions the ability to respond to certain requests.

So in the following example, I create `fastapi.fastAPI` under the name `my_first_app`, and create a function that will always respond `hello` to a `get` request.

In [20]:
%%writefile run_application_files/get_started.py
from fastapi import FastAPI

my_first_app = FastAPI()

@my_first_app.get("/")
def say_hello():
    return "hello"

Overwriting run_application_files/get_started.py


## Run

To run the `fastapi` official documentation recomend to use `uvicorn` as web-server. So you need to use command:

`uvicon <path to python file with program>:<name of fastapi.fastAPI object in your program>.`

So in the following example, a docker container is run, tested and stopped:

- Created from the image named `fastapi_experiment` described in the [docker file](#sec-dockerfile) above;
- With the name `test_container` for container;
- The default port for `fastapi` is 8000, so port 8000 on the container is connected to port 8000 on the local machine;
- With volume that allows to read [programme](#sec-prog);
- And with the command created from the required pattern:
    - `--host 0.0.0.0` is used to make the application visible from the container, it's not necessary if you are using `uvicorn` without docker.

In [ ]:
!docker run --rm -itd\
    --name test_container\
    -v ./run_application_files/get_started.py:/get_started.py\
    -p 8000:8000 \
    fastapi_experiment \
    uvicorn --host 0.0.0.0 get_started:my_first_app >/dev/null

So now you can try it in your browser, but here I use the `curl` utility - it returns "hello", just as I declared in the programme.

In [8]:
!curl localhost:8000

"hello"

And we can also check what is happening inside the container. **Note** that the last line here is the log line for the http request from the previous cell.

In [9]:
!docker logs test_container

INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     172.17.0.1:45308 - "GET / HTTP/1.1" 200 OK


**Don't forget** to stop the image when you've finished playing with the container.

In [ ]:
!docker stop test_container &> /dev/null

## Update program

The most convenient way to experiment with a container containing `fastapi` is to swap the program on the fly, so you can run many examples through one container. That's why I usually connect the used executing py file as a volume (so that changes on the computer get into the container at once).

But to implement it, you also need to run `uvicorn` with the `--reload` flag, which will make it track changes in the programme and update with it.

So, in the following cells:

- The container stats with a program that sends `initial line` as a response;
- Try in with `curl` - all right, got `initial line`;
- Then change the `reload.py` file to respond with `updated line`;
- Immediately try again with the same `curl` - we've got an `updated line` as response.

In [30]:
%%writefile run_application_files/reload.py
from fastapi import FastAPI

my_first_app = FastAPI()

@my_first_app.get("/")
def say_hello():
    return "initial line"

Overwriting run_application_files/reload.py


In [31]:
!docker run --rm -itd\
    --name test_container\
    -v ./run_application_files/reload.py:/reload.py\
    -p 8000:8000 \
    fastapi_experiment \
    uvicorn --host 0.0.0.0 --reload reload:my_first_app

752d9632c61a1927a227daf2a8b3de375385e107af1213a0df98c2b7d482d2ef


In [32]:
!curl localhost:8000

"initial line"

In [33]:
%%writefile run_application_files/reload.py
from fastapi import FastAPI

my_first_app = FastAPI()

@my_first_app.get("/")
def say_hello():
    return "updated line"

Overwriting run_application_files/reload.py


In [34]:
!curl localhost:8000

"updated line"

In [ ]:
!docker stop test_container

##